### Our favorite online shoe store, ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google. They want to know how the two ads are performing on each of the different platforms on each day of the week. Help them analyze the data using aggregate measures.

Examine the first few rows of ad_clicks.

In [3]:
import pandas as pd

ad_clicks = pd.read_csv('Documents/ad_clicks.csv')
print(ad_clicks.head())

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  


Your manager wants to know which ad platform is getting you the most views.

How many views (i.e., rows of the table) came from each utm_source?

In [5]:
ad_clicks.groupby('utm_source').user_id.count().reset_index().rename(columns={'user_id': 'count'})

,utm_source,count
0,email,255
1,facebook,504
2,google,680
3,twitter,215


If the column ad_click_timestamp is not null, then someone actually clicked on the ad that was displayed.

Create a new column called is_click, which is True if ad_click_timestamp is not null and False otherwise.

In [7]:
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()

We want to know the percent of people who clicked on ads from each utm_source.

Start by grouping by utm_source and is_click and counting the number of user_id‘s in each of those groups. Save your answer to the variable clicks_by_source.

In [9]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()

Now let’s pivot the data so that the columns are is_click (either True or False), the index is utm_source, and the values are user_id.

Save your results to the variable clicks_pivot.

In [11]:
clicks_pivot = clicks_by_source.pivot(
  columns = 'is_click',
  index = 'utm_source',
  values = 'user_id'
).reset_index()

Create a new column in clicks_pivot called percent_clicked which is equal to the percent of users who clicked on the ad from each utm_source.

Was there a difference in click rates for each source?

In [13]:
clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False])
print(clicks_pivot)

is_click utm_source  False  True  percent_clicked
0             email    175    80         0.313725
1          facebook    324   180         0.357143
2            google    441   239         0.351471
3           twitter    149    66         0.306977


The column experimental_group tells us whether the user was shown Ad A or Ad B.

Were approximately the same number of people shown both ads?

In [15]:
print(ad_clicks.groupby('experimental_group').\
user_id.count().reset_index())

  experimental_group  user_id
0                  A      827
1                  B      827


Using the column is_click that we defined earlier, check to see if a greater percentage of users clicked on Ad A or Ad B.

In [17]:
print(ad_clicks.groupby(['experimental_group', 'is_click']).\
user_id.count().reset_index())

  experimental_group  is_click  user_id
0                  A     False      517
1                  A      True      310
2                  B     False      572
3                  B      True      255


For each group (a_clicks and b_clicks), calculate the percent of users who clicked on the ad by day.


Compare the results for A and B. What happened over the course of the week

Do you recommend that your company use Ad A or Ad B?

In [19]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

a_clicks_day = a_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()
a_clicks_pivot = a_clicks_day.pivot(
  columns = 'is_click',
  index = 'day',
  values = 'user_id'
).reset_index()
a_clicks_pivot['percent_clicked'] = a_clicks_pivot[True]/(a_clicks_pivot[True] + a_clicks_pivot[False])

print(a_clicks_pivot)

b_clicks_day = b_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()
b_clicks_pivot = b_clicks_day.pivot(
  columns = 'is_click',
  index = 'day',
  values = 'user_id'
).reset_index()
b_clicks_pivot['percent_clicked'] = b_clicks_pivot[True]/(b_clicks_pivot[True] + b_clicks_pivot[False])

print(b_clicks_pivot)

is_click            day  False  True  percent_clicked
0            1 - Monday     70    43         0.380531
1           2 - Tuesday     76    43         0.361345
2         3 - Wednesday     86    38         0.306452
3          4 - Thursday     69    47         0.405172
4            5 - Friday     77    51         0.398438
5          6 - Saturday     73    45         0.381356
6            7 - Sunday     66    43         0.394495
is_click            day  False  True  percent_clicked
0            1 - Monday     81    32         0.283186
1           2 - Tuesday     74    45         0.378151
2         3 - Wednesday     89    35         0.282258
3          4 - Thursday     87    29         0.250000
4            5 - Friday     90    38         0.296875
5          6 - Saturday     76    42         0.355932
6            7 - Sunday     75    34         0.311927


My recommendation is to use ad A. It received more clicks overall and a higher percentage of clicks per day.